# 1. Positional Encodings:


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

plt.style.use('dark_background')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color = sns.color_palette('dark'))
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = (15 , 15)

In [12]:
# def positional_encoding(pos , dmodel):
#     res = []
#     if pos & 1:
#         for i in range(dmodel):
#             res.append(np.cos(pos / 10000 ** (i / dmodel)))
#     else:
#         for i in range(dmodel):
#             res.append(np.sin(pos / 10000 ** (i / dmodel)))

#     return np.array(res)

# #generate positional encodings for 10 positions and 4 dmodel embeddings:
# pencodings = []
# for i in range(20000):
#     pencodings.append(positional_encoding(i , 200))
# pencodings = np.array(pencodings)
# print(pencodings)
# sns.heatmap(pencodings)

In [13]:
#install dependencines:
# !pip3 install datasets transformers sentencepiece

# Sequence to sequence translation using transformers:


In [43]:
from datasets import load_dataset
from evaluate import load
from transformers import pipeline
import transformers

raw_datasets = load_dataset('wmt14' , 'de-en')

In [48]:
raw_datasets['train']['translation'][10:18:2]

[{'de': 'Wäre es angemessen, wenn Sie, Frau Präsidentin, der Präsidentin von Sri Lanka in einem Schreiben das Bedauern des Parlaments zum gewaltsamen Tod von Herrn Ponnambalam und anderen Bürgern von Sri Lanka übermitteln und sie auffordern würden, alles in ihrem Kräften stehende zu tun, um nach einer friedlichen Lösung dieser sehr schwierigen Situation zu suchen?',
  'en': "Would it be appropriate for you, Madam President, to write a letter to the Sri Lankan President expressing Parliament's regret at his and the other violent deaths in Sri Lanka and urging her to do everything she possibly can to seek a peaceful reconciliation to a very difficult situation?"},
 {'de': 'Wenn das Haus damit einverstanden ist, werde ich dem Vorschlag von Herrn Evans folgen.',
  'en': 'If the House agrees, I shall do as Mr Evans has suggested.'},
 {'de': 'Könnten Sie mir eine Auskunft zu Artikel 143 im Zusammenhang mit der Unzulässigkeit geben?',
  'en': 'I would like your advice about Rule 143 concernin

In [24]:
split_data = raw_datasets['train'].train_test_split(train_size = 0.9 , seed = 20)

#rename test key to validation:
split_data['validation'] = split_data.pop('test')
split_data

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4057906
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 450879
    })
})

In [61]:
model_en_de = 'Helsinki-NLP/opus-mt-en-de'
model_de_en = 'Helsinki-NLP/opus-mt-de-en'
translator = pipeline('translation' , model = model_de_en , framework = 'pt')
translator2 = pipeline('translation' , model = model_en_de , framework = 'pt')

/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use mps:0
Device set to use mps:0


In [62]:
print(f"Translating German to English")
i = 4
tdata = split_data['train']['translation'][i]
ttranslation = translator(tdata['de'])
print(f"German sentence: {tdata['de']}")
print(f"Translated sentence: {ttranslation[0]['translation_text']}")
print(f"Original text: {tdata['en']}")

Translating German to English
German sentence: Zeitschriften wie Multitudes können fast ausschließlich auf der Basis unentgeltlicher Zusammenarbeit produziert werden.
Translated sentence: Journals such as multitudes can be produced almost exclusively on the basis of free cooperation.
Original text: A journal like Multitudes can be made almost entirely through unpaid cooperation.


In [63]:
print(f"Translating English to German:")
i = 4
tdata = split_data['train']['translation'][i]
ttranslation = translator2(tdata['en'])
print(f"English sentence: {tdata['en']}")
print(f"Translated sentence: {ttranslation[0]['translation_text']}")
print(f"Original text: {tdata['de']}")

Translating English to German:
English sentence: A journal like Multitudes can be made almost entirely through unpaid cooperation.
Translated sentence: Eine Zeitschrift wie Multitudes kann fast vollständig durch unbezahlte Zusammenarbeit erstellt werden.
Original text: Zeitschriften wie Multitudes können fast ausschließlich auf der Basis unentgeltlicher Zusammenarbeit produziert werden.


In [66]:
#load tokenizer:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_de_en , return_tensors = 'pt')

/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [73]:
#pre-processing a single sample:
i = 4
german_sentence = split_data['train'][i]['translation']['de']
english_sentence = split_data['train'][i]['translation']['en']

inputs = tokenizer(german_sentence)

with tokenizer.as_target_tokenizer():
    targets = tokenizer(english_sentence)

wrong_targets = tokenizer(english_sentence)
print(f"{tokenizer.convert_ids_to_tokens(wrong_targets['input_ids'])}")
print(f"{tokenizer.convert_ids_to_tokens(targets['input_ids'])}")

['▁A', '▁', 'journal', '▁li', 'ke', '▁Multi', 'tu', 'des', '▁can', '▁be', '▁', 'made', '▁al', 'mo', 'st', '▁ent', 'ire', 'ly', '▁', 'thro', 'ugh', '▁un', 'pa', 'id', '▁co', 'operation', '.', '</s>']
['▁A', '▁journal', '▁like', '▁Multi', 'tu', 'des', '▁can', '▁be', '▁made', '▁almost', '▁entirely', '▁through', '▁unpaid', '▁cooperation', '.', '</s>']


In [76]:
split_data['train']['translation']

Column([{'de': 'Zur Frage, ob die Kommission auf der Konferenz in Kampala Änderungsanträge vorlegen wird: die Antwort hier ist "nein", weil die EU als solche keine Vertragspartei ist - es sind die Mitgliedstaaten, die die Änderungsanträge verhandeln werden.', 'en': "On the question of whether the Commission will present amendments in the Kampala meeting, here the answer is 'no' because the EU, as such, is not a party - it is the Member States who will negotiate the amendments."}, {'de': 'Der Rat hat in der Vergangenheit versucht und wird es auch in Zukunft tun, politische Reformen zu fördern, und zwar durch Unterstützung verschiedener Stellen und Organisationen im Iran, einschließlich der iranischen Zivilgesellschaft.', 'en': 'The Council has endeavoured in the past and will endeavour in future to promote political reform, namely by supporting various authorities and organisations in Iran, including Iranian civil society.'}, {'de': 'Hier finden Sie aktuelle Nachrichten rund um die Firm